<a href="https://colab.research.google.com/github/lapshinaaa/recsys-tasks/blob/main/RecSys3_CanGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Systems
## Candidate Generation

Note from YSDA course: During practice, we've discussed common CG stage abstractions, quality metrics and implemented several models with a KNN candidate generator class. In homework your task will be to dive deeper into this domain and implement a few more models, compare them and come up with a way to use all of them simultaneously. The homework consists of several subtasks, which are independent, and two bonus subtasks.

Let's go!

### Imports and metrics setup
We'll use the metrics and evaluator class, which we were using in practice about candidate generators.

In [1]:
# if you are running this notebook in colab, datasphere, etc - you need to clone the repository first

!git clone https://github.com/yandexdataschool/recsys_course
!pip install -r recsys_course/week03/homework/requirements.txt
!pip install -e recsys_course/grocery

Cloning into 'recsys_course'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 206 (delta 8), reused 5 (delta 4), pack-reused 189 (from 1)
Receiving objects: 100% (206/206), 34.65 MiB | 36.21 MiB/s, done.
Resolving deltas: 100% (75/75), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'recsys_course/week03/homework/requirements.txt'
Obtaining file:///content/recsys_course/grocery
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 416.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 130.6 MB/s eta 

In [1]:
import numba as nb
import numpy as np
import polars as pl
import scipy.sparse as sp
from sklearn.model_selection import train_test_split

from grocery.metrics import Evaluator, Recall, Novelty, CategoryDiversity
from grocery.recommender.candidates import CandidateGenerator, Candidate
from grocery.utils.dataset import build_matrix_with_mappings

In [5]:
from scipy.sparse.linalg import inv

In [2]:
DATA_DIR = "../../data/lavka"

from grocery.utils.dataset import download_and_extract
download_and_extract(
     url="https://www.kaggle.com/api/v1/datasets/download/thekabeton/ysda-recsys-2025-lavka-dataset",
     filename="lavka.zip",
     dest_dir=DATA_DIR
 )

lavka.zip: 100%|██████████████████████████████████████████████████| 447M/447M [00:11<00:00, 37.6MB/s]


Unpacking lavka.zip...
Files from lavka.zip successfully unpacked



In [3]:
data = (
    pl.read_parquet(f"{DATA_DIR}/train.parquet")
    .filter(pl.col("action_type") == "AT_CartUpdate")
    .select(
        pl.col("user_id"),
        pl.col("product_id").alias("item_id"),
        pl.col("request_id"),
        pl.lit(1).alias("rating"),
        pl.col("timestamp"),
        pl.col("product_category"),
    )
)

# global timepoint split!
train, test = train_test_split(data.sort("timestamp"), test_size=0.2, shuffle=False)

In [4]:
evaluator = Evaluator(
    metrics=[
        Recall(10),
        Recall(50),
        Novelty(train, 10),
        Novelty(train, 50),
        CategoryDiversity(train, 10),
        CategoryDiversity(train, 50),
    ]
)

evaluator.load_test_actions(
    test.filter(pl.col("user_id").is_in(train["user_id"].unique()))
)

## Candidate Generators

### EASE (0.5 points), LinearItemToItemCG and ANN (1 points)
Implement the model from [Embarrassingly Shallow Autoencoders for Sparse Data](https://arxiv.org/pdf/1905.03375).

From practice we know that we can present our training data as a sparse matrix $X \in \mathbb{R}^{|U| x |I|}$, where each element $x_{ui}$ is a binary value, which is positive if the user $u$ had an interaction with item $i$, or rating from their interactions, or number of interactions.

EASE suggests a linear model, that encodes all items as an item-item matrix $B$, and each item can be represented as a linear combination of other items.
We consider two features. The first is categorical — the ID of the item being evaluated. The second group consists of items with which the user has had positive interactions (excluding the item in question). We then take the cross-product of these two groups to generate features like: [currently evaluating item i, item j appears in the user’s history]. We train a linear model on these cross-features. The target is 1 for positive interactions and 0 otherwise (even if the user hasn’t been shown the item).

We use MSE loss and L2 regularization (a convex loss function), adding the constraint that diagonal weights must be zero. This constraint, $diag(B) = 0$, is crucial as to avoid the trivial solution $B = I$ (self-similarity of items), where $I$ is the identity matrix.

The optimization problem in matrix form is as follows:

$$\min_{B} ||X - XB||^2_F + \lambda ||B||^2_F$$
$$\text{s.t. diag}(B) = 0$$

Turns out, the problem has an exact analytical solution - your task is to derive it and implement!

*Notes: it's better to implement both sparse with scipy.sparse.linalg.inv approach and dense with np.linalg.inv, and to check if dense matrix fits in your memory :-)*

In [7]:
class EASE:
    def __init__(self, lambd: float = 0.1):
        self.lambd = lambd
        self.B = None
        self.item_id2idx = None # column index -> item_id
        self.item_idx2id = None # item_id -> column index

    def fit(self, ratings: pl.DataFrame, sparse_inv: bool = False):
        X, (user_id2idx, item_id2idx, user_idx2id, item_idx2id) = build_matrix_with_mappings(ratings)
        self.item_id2idx = item_id2idx
        self.item_idx2id = item_idx2id

        # Gram Matrix G (according to the article) = X^T X
        G = X.T @ X  # shape: (num_items x num_items), sparse

        # λ * I for regularization
        num_items = G.shape[0]
        G = G + self.lambd * sp.eye(num_items, format="csc")

        P = inv(G)  # returns sparse linear operator
        P = P.toarray()  # convert to dense (safe because only num_items x num_items)

        #  B
        B = -P / np.diag(P)  # broadcasting division by diagonal
        np.fill_diagonal(B, 0)  # enforce diag(B)=0

        self.B = B
        return self

Now let's implement a class to use that model for extracting candidates (or straight-up recommending) for a certain user.

Some notes:
- Do not forget about the fact that model operates in indices, but CG should receive user IDs and return `Candidates` with item IDs (not indices!)
- Utilize the fact that user-item interactions is a sparse matrix, meaning there are only few positive elements, so you can try to iterate over them, extract needed weights from sparse matrix and sum them.
- You may not even store the user-item matrix, but receive a dict with user history as a list of IDs during `__init__.py`. Although this is not a part of this homework, such class would later allow you to switch the dict for some other user history storage (or maybe even a generator, yeilding results from a database, .etc).

In [8]:
class LinearItemToItemCG(CandidateGenerator):
    def __init__(self,
                 B: sp.spmatrix | np.ndarray,
                 user_id2idx: dict[int, int],
                 item_id2idx: dict[int, int],
                 X: sp.spmatrix | np.ndarray,
                #  history: dict[int, list[int]],
                 ):
        """Item to Item Linear Model applier.

        Args:
            B (sp.sparse.spmatrix | np.ndarray): Weight matrix.
            user_id2idx (dict[int, int]): Mapping from user IDs to indices.
            item_id2idx (dict[int, int]): Mapping from item IDs to indices.
            X (sp.sparse.spmatrix | np.ndarray): User-item interaction matrix.
        """
        super().__init__()

        self.B = B                          # (num_items × num_items)
        self.user_id2idx = user_id2idx      # map user_id → row index in X
        self.item_id2idx = item_id2idx      # map item_id → column index in X
        self.idx2item_id = {v: k for k, v in item_id2idx.items()} # reversing the mapping here
        self.X = X


    def extract_candidates(self, object_id: int, n: int = 10) -> list[Candidate]:

        if object_id not in self.user_id2idx: # edge case user does not exist
          return []

        user_idx = self.user_id_2idx[object_id]
        user_vector = self.X[user_idx] # getting the user history as a sparse row

        # now we're going to score the items and remove the ones the user has already interacted with
        scores = user_vector @ self.B
        scores = np.asarray(scores).ravel()
        seen_items = set(self.X[user_idx].indices)
        scores[list(seen_items)] = -np.inf # we're not recommending what has already been seen

        # select top items
        top_indices = np.argpartition(-scores, n)[:n]
        top_indices = top_indices[np.argsort(-scores[top_indices])]

        return [Candidate(id=self.idx2id[idx]) for idx in top_indices]

Your model should meet the quality level needed - for defined in this notebook train dataset, initialization and default hyperparameters from model class template, reference implementation achieves:

$$recall@10 >= 0.04$$
$$recall@50 >= 0.15$$

In [ ]:
model = EASE(lambd=0.1)
model.fit(train)

ease_cg = LinearItemToItemCG(model.B, model.item_id2idx, model.item_id2idx, X)

metrics = evaluator.evaluate(lambda user_id, n: ease_cg.extract_candidates(user_id, n))
for metric_name, metric_value in metrics.items():
    print(f"{metric_name} = {metric_value:.3f}")

Now let's implement ANN candgen and compare searching in ALS embedding space with EASE scoring. There are lots of options to choose from, but we recommend settling on `faiss` or `voyager` libraries. The latter utilizes HNSW algorithm and has `ef_construction` parameter when building and `query_ef` parameter in `query` method, they control the tradeoff between speed and recall - feel free to experiment a bit with them.

Here I could say *"you can write your own HNSW"*, but this is breaking a butterfly upon the wheel - please don't. The fact that ANN is used instead of KNN is enough optimization :D

In [ ]:
from grocery.models import ALS
from grocery.recommender.candidates import DotProductKNN

def extract_embeddings(model: ALS) -> tuple[dict[int, np.array], dict[int, np.array]]:
    return (
        {ID: model.user_vectors[idx] for ID, idx in model.user_id2idx.items()},
        {ID: model.item_vectors[idx] for ID, idx in model.item_id2idx.items()}
    )

In [ ]:
from voyager import Index, Space
# what space do you need to use?


class ANN(CandidateGenerator):
    def __init__(self,
                 left_embeddings: dict[int, np.array],
                 right_embeddings: dict[int, np.array]):
        super().__init__()

        # TODO: your code here

        pass

    @staticmethod
    def build_index(vectors: np.ndarray, ids: list[int]):

        # TODO: your code here

        pass

    def extract_candidates(self, object_id: int, n: int = 10) -> list[Candidate]:

        # TODO: your code here

        pass


Compare the resulting class with DotProductKNN from practice: speed, recall.

In [ ]:
als = ALS(max_iter=100)
als.fit(train)

user_embeddings, item_embeddings = extract_embeddings(als)
als_ann = ANN(user_embeddings, item_embeddings)
als_knn = DotProductKNN(user_embeddings, item_embeddings)

# TODO: your code here
# compare speed and recall, better to write your thoughts as md cell

### SLIM (1 point)

The next model in our arsenal will be **S**parse **Li**near **M**ethod a.k.a. [SLIM](https://ieeexplore.ieee.org/document/6137254) (PDF is available [here](https://www.researchgate.net/profile/George_Karypis/publication/220765374_SLIM_Sparse_Linear_Methods_for_Top-N_Recommender_Systems/links/549ee9ac0cf257a635fe7010.pdf)).

Actually, SLIM was introduced long before EASE, and EASE cites SLIM (and uses a lot of it as a foundation), not the other way around. For educational purposes, we switched their order of presentation in the homework. The algorithm solves the same problem of finding best item-item weight matrix, but with different loss - the main difference is L1 regularization for sparsity component, which helps the algorithm to build a sparse weight matrix, which is both memory-efficient and compute-efficient, if you organize your storage accordingly.


The optimization problem:

$$\min_{B} \frac{1}{2}||X - XB||^2_F + \frac{\beta}{2} ||B||^2_F + \lambda ||B||_F$$
$$\text{subject to } B \geq 0 \text{, diag}(B) = 0$$

This is solvable with coordinate descent by projecting each individual weight (non-negative constraint) and manually setting diagonal weights to zero during all the steps. Moreover, each item weight vector is independent from others, so the task is easily parallelizable across items. However, there aren't any optimizers that can do that out of the box. As proposed in this [paper](https://www.slideshare.net/slideshow/efficient-slides/27138952), we'll simplify the problem a bit by dropping the $diag(B) = 0$ constraint and setting the item column of the interaction matrix to zero when fitting for it. Let's see, whether it's still a viable model - simple scikit-learn should do the trick.

Implement SLIM model using sklearn ElasticNet solver and optionally multiprocessing. The steps are as follows:
- Build sparse matrix of interactions
- For each item $u$:
    - Get interaction matrix $X$ - the features
    - Extract target vector $y = X_u \in R^{|U|}$ - all interactions with this item
    - Find solution and save it
- Extract sparse coefs to one weight matrix B

*Notes:* multiprocessing and jupyter notebooks are not really friends, so if you want to speed up the model training by parallelization - move it into a separate file, save matrix to `.npy` and launch it from here using Ipython magic.

In [ ]:
from tqdm import trange
from sklearn.linear_model import ElasticNet


def _solve_item(X, item_idx, alpha, l1_ratio):

    # TODO: your code here

    # X, y = ...
    # solver = ElasticNet(...)
    # solver.fit(...)
    # coef = ...

    pass


class SLIM:
    def __init__(self,
                 alpha: float = 0.1,
                 l1_ratio: float = 0.01,
                 num_processes: int | None = None,
                 ):

        # TODO: your code here

        pass

    def fit(self, ratings: pl.DataFrame):

        # TODO: your code here

        # self.B = sp.sparse.matrix or smth like that
        # ... = build_matrix_with_mappings(ratings)
        # for item in range(num_items):
        #     self.B[item] = _solve_item(...)

        pass

In [ ]:
slim = SLIM(num_processes=1)
slim.fit(train)

Your model should meet the quality level needed - for defined in this notebook train dataset, initialization and default hyperparameters from model class template, reference implementation achieves:

$$recall@10 >= 0.04$$
$$recall@50 >= 0.12$$

In [ ]:
slim_cg = LinearItemToItemCG(...)

# TODO: your code here
# compare speed and recall, better to write your thoughts as md cell

#### Bonus task
If you're really interested in implementing the models from scratch, you can try to implement your own SLIM using a lower-level language with parallelization. Here are some hints on how you can build and speed up your implementation.

The proposed way is to use the coordinate descent, so if we fix item $i$ - we can solve the task independently:

$$ L_i = \frac{1}{2} || a_i - \sum_j w_{ij}a_j || ^ 2 + \lambda \sum_j |w_{ij}| + \beta \sum (w_{ij}^2) \rightarrow \min_{w_{i1}...w_{in}}$$

where $a_j$ is j-th column of interaction matrix. Since we are solving for nonnegative weights, all the modules of weights can be removed.

The step of coordinate descent consists of fixating all the weights except one and fully optimizing by one component. The equation for the weight optimum is dervied from solving the equation of gradient being equal to zero:

$$w_{ik} = \frac{\langle a_i, a_k \rangle - \sum_{j \ne k}w_{ij} \langle a_j, a_k \rangle - \lambda}{||a_k||^2 + \beta}$$

For diagonal weight, we simply set the weight to zero, and for all the other weights we project in onto the feasible set:

$$w_{ik} = ReLU \left(\frac{\langle a_i, a_k \rangle - \sum_{j \ne k}w_{ij} \langle a_j, a_k \rangle - \lambda}{||a_k||^2 + \beta} \right)$$

In case of binary interaction matrix, if you look closely, there are some iterations that you can skip because for certain $\lambda$ value the constraints make the weights go to zero in any scenario. If $\langle a_j, a_k \rangle < \lambda$ - you can skip the iterations with number $k$; if for item $i$ the number of interactions is less than $\lambda$ - you can just skip this item altogether and set them to zero.

Good luck, if you decide to tackle this! Working solution will get you up to 2 points, if you manage to present:
- the source code (yours).
- setup and run instructions.
- resulting recall (you can get it from loading the matrix into **LinearItemToItemCG** class).

### BPR with popularity sampling (1.5 points)

Now let's try to improve the matrix factorization models. In practice we tested alternating least squares and logistic factorization, which were trying to solve an optimization problem of predicting the correct score. But in recommender systems, we usually need to rank the items, and that is exactly what [BPR: Bayesian Personalized Ranking from Implicit Feedback](https://arxiv.org/pdf/1205.2618) does. Instead of predicting explicit ratings, BPR assumes that a user prefers items with which they have interacted over those they have not. It optimizes a pairwise ranking objective to ensure that, for each user, positive (observed) items are ranked higher than negative (unobserved) items, using gradient-based optimization (typically SGD, but there implementations that allow RMSProp, Adam, etc).

The BPR loss aims to maximize the probability that for a given user $u$, a positively interacted item $i$ is ranked higher than a negative item $j$. So, for a triplet $u, i, j$ the loss is defined as:

$$L(u, i, j) = -\ln \sigma(\hat{x}_{ui} - \hat{x}_{uj}) + \lambda ||\Theta||^2$$

where $\Theta$ represents parameters and adds for regularization. In our task, you'll be using a matrix factorization approach with user biases:

$$\hat{x}_{ui} = \langle p_u, q_u \rangle + b_{i}$$

##### Optimizer step

First, implement the optimizer step for a triplet $u, i, j$ - this will be your mini-batch in SGD optimization. For sampled indices, update the user embedding, item embedding and item bias. The derivation of the gradients for parameters is left to you (and is relatively easy to do).

Both in this and the next task reference solution contains some parts, that are JIT-compiled using `Numba` and seems to be working significantly faster. It is recommended to also adapt your code so that it's suitable for `Numba` (you'll have the templates for functions), but if you're lazy and patient - you can just send a pure Python version. But you still need to fully train the model at least once!

In [ ]:
@nb.njit
def optimizer_step(user_vectors, item_vectors, user_idx, pos_item_idx, neg_item_idx, lr, lambd):
    """Optimization step for BPR model
    1. Calculate the dot-product scores for the positive and negative items.
    2. Apply sigmoid function to the difference of the scores to get P(u, i > j).
    3. Calculate the gradients for the user and item vectors.
    4. Update inplace the user and item vectors.

    Args:
        user_vectors (np.ndarray[n_users, dim]): User embeddings.
        item_vectors (np.ndarray[n_items, dim]): Item embeddings.
        user_idx (int): User index.
        pos_item_idx (int): Positive item index.
        neg_item_idx (int): Negative item index.
        lr (float): Learning rate.
        lambd (float): Regularization coefficient.
    """

    # TODO: your code here

    pass

In [ ]:
def test_bpr_optimizer(step_function):
    user_embeddings = np.array([
        [-0.004, -0.01 , -0.012,  0.038,  0.054],
        [ 0.012,  0.038,  0.   , -0.06 ,  0.015]
    ], dtype=np.float32)

    item_embeddings = np.array([
        [ 0.034,  0.047,  0.061, -0.043, -0.017],
        [-0.005,  0.065, -0.086,  0.055, -0.013]
    ], dtype=np.float32)

    gt_user_embeddings = np.array([
        [-0.00200446, -0.01080256, -0.00450913,  0.03270608,  0.05325943],
        [ 0.012     ,  0.038     ,  0.        , -0.06      ,  0.015     ],
    ], dtype=np.float32)

    gt_item_embeddings = np.array([
        [ 0.03345943,  0.04602858,  0.0597883,  -0.04066461, -0.01412233],
        [-0.00474943,  0.06485142, -0.0845383,   0.0525446,  -0.01557767],
    ], dtype=np.float32)

    optimizer_step(user_embeddings, item_embeddings, 0, 0, 1, 0.1, 0.1)

    assert np.allclose(user_embeddings, gt_user_embeddings), f"User embeddings are not close to the ground truth"
    assert np.allclose(item_embeddings, gt_item_embeddings), f"Item embeddings are not close to the ground truth"

    user_embeddings = np.array([
        [1, 0, 0, 0, 0],
        [1, 1, -1, -1, 0]
    ], dtype=np.float32)

    item_embeddings = np.array([
        [0, 1, 0, -1, 1],
        [-1, 0, 1, 0, -1]
    ], dtype=np.float32)

    gt_user_embeddings = np.array([
        [ 1.2689414 ,  0.26894143, -0.26894143, -0.26894143,  0.53788286],
        [ 1.        ,  1.        , -1.        , -1.        ,  0.        ]
    ], dtype=np.float32)

    gt_item_embeddings = np.array([
        [ 0.26894143,  1.        ,  0.        , -1.        ,  1.        ],
        [-1.2689414 ,  0.        ,  1.        ,  0.        , -1.        ]
    ], dtype=np.float32)

    step_function(user_embeddings, item_embeddings, 0, 0, 1, 1, 0)

    assert np.allclose(user_embeddings, gt_user_embeddings), f"User embeddings are not close to the ground truth"
    assert np.allclose(item_embeddings, gt_item_embeddings), f"Item embeddings are not close to the ground truth"

In [ ]:
test_bpr_optimizer(optimizer_step)

##### Sampling
The next step is to implement the negative sampling. We will be storing IDs in the same format, as they are in dataset - just two arrays of IDs, where I-th interaction was between `user_ids[i]` and `item_ids[i]`. This will allow us to sample both users and items based on their "popularity" - the more times item or user was interacting with something, more frequently it will pop up in triplets. It's a good heuristic and works well in BPR.

In proposed implementation, we will be sampling any item as negative and then checking if it's actually negative with lookup into all user positives. Because there are a lot of items and dataset is sparse, the chance of sampling a fake negative is not really high, so we won't be slipping too much. Depending on implementation, storing all the positives, negatives and sampling from them may be slower or faster, but this one is definitely fast!

In [ ]:
@nb.njit
def sample_triplet_popularity(item_ids: np.ndarray, user_ids: np.ndarray):
    """Sample triplet using fast popularity sampling from whole dataset, represented
    as two arrays of item and user IDs for each positive interaction from the dataset.
    1. Sample random interaction of the dataset
    2. Get user ID, positive item ID from this interaction.
    3. Sample random interaction of the dataset as negative item.
    The approach does not guarantee that the negative item is indeed negative for a user.

    Args:
        item_ids (np.ndarray[n_samples]): array of item IDs, each point represents a positive interaction from the dataset.
        user_ids (np.ndarray[n_samples]): array of user IDs, each point represents a positive interaction from the dataset.

    Returns:
        tuple[int, int, int]: user ID, positive item ID, negative item ID.
    """

    # TODO: your code here
    # return user_id, positive_id, negative_id

    pass

In [ ]:
def test_sample_triplet_popularity(sampler, num_samples=100000):
    item_ids = [4, 1, 2, 4, 3, 3, 4, 4, 2, 3]
    user_ids = [1, 1, 2, 2, 3, 3, 4, 4, 3, 3]
    # ratings data would look something like this
    # pl.DataFrame({
    #     "user_id": [4, 1, 2, 4, 3, 3, 4, 4, 5, 5],
    #     "item_id": [1, 1, 2, 2, 3, 3, 4, 4, 5, 5],
    # })

    sampled_triplets = [sampler(item_ids, user_ids) for _ in range(num_samples)]
    users, positives, negatives = zip(*sampled_triplets)

    gt_user_freqs = np.bincount(user_ids) / len(user_ids)
    gt_item_freqs = np.bincount(item_ids) / len(item_ids)
    user_freqs = np.bincount(users) / len(users)
    pos_item_freqs = np.bincount(positives) / len(positives)
    neg_item_freqs = np.bincount(negatives) / len(negatives)

    assert np.allclose(gt_user_freqs, user_freqs, atol=0.01)
    assert np.allclose(gt_item_freqs, pos_item_freqs, atol=0.01)
    assert np.allclose(gt_item_freqs, neg_item_freqs, atol=0.01)

In [ ]:
test_sample_triplet_popularity(sample_triplet_popularity)

##### The actual model
Some code hints are left as comments, and docstring should provide the needed guidelines on how to implement the methods. Feel free to change the implementation if you want, as long as it's still BPR and it meets the quality benchmarks!

In [ ]:
from tqdm import trange
from collections import defaultdict


class PopularityBPR:
    def __init__(self,
                 dim: int = 128,
                 max_iter: int = 100,
                 learning_rate: float = 0.01,
                 lambd: float = 0.01,
                ):
        self.dim = dim
        self.max_iter = max_iter
        self.learning_rate = learning_rate
        self.lambd = lambd

    def _init_parameters(self, ratings: pl.DataFrame):
        """Initialize parameters for the model.
        1. Build mappings from user and item IDs to indices
        2. Initialize user and item vectors with scaled normal distribution
        3. Initialize item biases with zeros
        4. Set item biases to zero if there are no interactions for an item
        5. Set user vectors to zero if there are no interactions for a user

        Args:
            ratings (pl.DataFrame): pl.DataFrame with ratings
        """
        R, mappings = build_matrix_with_mappings(ratings, additive=False)
        self.user_id2idx, self.item_id2idx, self.user_idx2id, self.item_idx2id = mappings
        self.num_samples = len(ratings)
        self.n_users = len(self.user_id2idx)
        self.n_items = len(self.item_id2idx)
        self.item_biases = np.zeros((self.n_items, 1))
        self.user_vectors = np.random.normal(loc=0, scale=0.2 / self.dim, size=(self.n_users, self.dim))
        self.item_vectors = np.random.normal(loc=0, scale=0.2 / self.dim, size=(self.n_items, self.dim))
        user_has_no_interactions = R.sum(axis=1) == 0
        item_has_no_interactions = R.sum(axis=0) == 0
        self.item_vectors[item_has_no_interactions] = 0
        self.user_vectors[user_has_no_interactions] = 0


    def _init_sampler(self, ratings: pl.DataFrame) -> tuple[np.ndarray, np.ndarray]:
        """
        Convert ratings to the format, awaited by the sampling function - meaning
        np.arrays of item and user IDs for each positive interaction from the dataset.
        Also save the user positives in any lookup-friendly data structure for checking,
        whether and item is negative or positive for a user to skip it after sampling.

        Args:
            ratings (pl.DataFrame): Ratings data in standard format.
        Returns:
            tuple[np.ndarray, np.ndarray]: Item IDs and user IDs.
        """

        # TODO: your code here

        pass

    def train_one_epoch(self):
        """Train one epoch of the model and compute the number of skipped triplets.
        For num_samples times:
        1. Sample a triplet (user, positive item, negative item)
        2. If the negative item is not a true negative for the user, skip the triplet
        3. Otherwise, update the model parameters

        Returns:
            int: Number of skipped triplets.
        """

        # TODO: your code here

        # skipped = 0
        # for _ in range(self.num_samples):
        #       triplet = sample_triplet()
        #       if verify_triplet(triplet):
        #           optimizer_step(triplet)
        # return skipped

        pass

    def fit(self, ratings: pl.DataFrame):
        """Fit the model to the ratings data.
        1. Initialize parameters and sampler
        2. Train the model for max_iter epochs
        3. Print the number of skipped triplets

        Args:
            ratings (pl.DataFrame): Ratings data in standard format.
        """

        # TODO: your code here

        pass

In [ ]:
bpr = PopularityBPR(max_iter=50)
bpr.fit(train)

For defined in this notebook train dataset, initialization and default hyperparameters from model class template, reference implementation achieves:

$$recall@10 >= 0.07$$
$$recall@50 >= 0.18$$

In [ ]:
user_embeddings, item_embeddings = extract_embeddings(bpr)
knn = DotProductKNN(user_embeddings, item_embeddings)

# TODO: your code here
# compare speed and recall, better to write your thoughts as md cell

### BPR with adaptive sampling (1.5 points)

Now let's spice things up a bit!

Popularity sampling is fast and effective, but eventually it converges to a state where it primarily samples easy, obvious negatives and does not learn anything new. This can be fixed by using hard negatives - the ones, that are negative for user, but have high model score. We'll do this by changing popularity sampling to weighted sampling, where weights are proportionate to model confidence.

For positive weights, we'll compute the probabilites as frequencies in dataset for each user (or any other way if you want to). For first `num_epochs_without_updates` we'll set the negative weights uniform, after that we'll compute the scores and softmax them with temperature to get the sampling probabilities. The higher the score - the more frequently we will be sampling these negatives.

Since we're using complex weighted sampling (three random generator calls in total), we'll ensure that both positive and negative weights are correct, meaning that negative items have zero weight in positive sampling weights and vice versa. This should be done both in sampler initialization and during weights update.

In [ ]:
@nb.njit
def nb_random_choice(arr, prob):
    """Analogue of weighted np.random.choice (Numba-compatible).
    Args:
        arr (np.ndarray): Array of items to sample from.
        prob (np.ndarray): Array of probabilities for each item.
    Returns:
        int: Index of the sampled item.
    """
    return arr[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]


@nb.njit
def weighted_sample_triplet(users, items, user_weights, positive_weights, negative_weights):
    """Weighted sampling of triplet (user, positive item, negative item).

    Args:
        users (np.ndarray): Array of user IDs.
        items (np.ndarray): Array of item IDs.
        user_weights (np.ndarray[n_users]): Array of user weights.
        positive_weights (np.ndarray[n_users, n_items]): Array of positive item weights for each user.
        negative_weights (np.ndarray[n_users, n_items]): Array of negative item weights for each user.

    Returns:
        tuple[int, int, int]: User ID, positive item ID, negative item ID.
    """
    user_idx = nb_random_choice(users, user_weights)
    positive_item_idx = nb_random_choice(items, positive_weights[user_idx])
    negative_item_idx = nb_random_choice(items, negative_weights[user_idx])
    return user_idx, positive_item_idx, negative_item_idx

In [ ]:
class AdaptiveBPR(PopularityBPR):
    def __init__(self,
                 dim: int = 128,
                 max_iter: int = 100,
                 learning_rate: float = 0.01,
                 lambd: float = 0.01,
                 weight_temperature: float = 0.5,
                 num_epochs_without_updates: int = 10
                ):
        super().__init__(dim, max_iter, learning_rate, lambd)
        self.t = weight_temperature
        self.num_epochs_without_updates = num_epochs_without_updates

    def _init_sampler(self, ratings: pl.DataFrame):
        """Initialize sampler weights.
        There should be three weights, all proportional to popularity in dataset:
        1. User weights - np.array[n_users]
        2. Positive item weights - np.array[n_users, n_items]
        3. Negative item weights - np.array[n_users, n_items]
        Do not forget to normalize item weights after initialization inside each row and all user weights.

        Args:
            ratings (pl.DataFrame): Ratings data in standard format.
        """

        # TODO: your code here

        # user_weights = [0, 0, 0, ...]
        # positive_weights = [[0, 0, 0, ...], [0, 0, 0, ...], ...]

        # for user, item in dataset:
        #     user_weights[user] += 1
        #     positive_weights[user, item] += 1

        pass

    def train_one_epoch(self):
        """
        For num_samples times:
        1. Sample a triplet (user, positive item, negative item) using weighted negative sampling
        3. Update the model parameters

        Returns:
            int: Number of skipped triplets.
        """

        # TODO: your code here

        pass

    def update_weights(self):
        """Update negative item weights based on model scores.
        1. Compute the scores for all items for each user.
        2. Set to zero the weights for positive items, that are not negative for the user.
        3. Apply softmax with temperature to the negative weights.
        """

        # TODO: your code here

        # scores = user_emb @ item_emb
        # negative_mask = self.negative_weights > 0
        # negative_weights = softmax(scores[negative_mask], t)

        pass

    def fit(self, ratings: pl.DataFrame):
        """Fit the model to the ratings data.
        - First initialize parameters
        - Initialize sampler and weights
        - For max_iter epochs:
            - Train the model for one epoch
            - If epoch >= num_epochs_without_updates, update the weights

        Args:
            ratings (pl.DataFrame): Ratings data in standard format.
        """

        # TODO: your code here

        pass

In [ ]:
bpr = AdaptiveBPR()
bpr.fit(train)

For defined in this notebook train dataset, initialization and default hyperparameters from model class template, reference implementation achieves:

$$recall@10 >= 0.09$$
$$recall@50 >= 0.20$$

In [ ]:
user_embeddings, item_embeddings = extract_embeddings(bpr)
knn = DotProductKNN(user_embeddings, item_embeddings)

# TODO: your code here
# compare speed and recall, better to write your thoughts as md cell

### Quality benchmark (0.5 points)

This part is supposed to inspire you to be creative!

Come up with a benchmark setup, where you compare:
- The speed of model training on different dataset sizes (at least three).
- The memory of stored CGs (ANN index, `.npz` file or sparse/dense I2I matrix).
- The speed of CGs on inference (request per second in non-batched format).
- The $recall@10$ and $recall@50$ of the algorithms you got.

Each metric gets you 0.1 point, except model training speed which gets you 0.2 points. Present your results in a human-readable format, preferably using pandas/polars DataFrames or various plots within the notebook. Reports in an unreadable format may receive fewer points.

In [ ]:
# TODO: your code here

## Bonus task: MixiGen

MixiGen is Yandex's approach to create personalized adaptive blending of candidate generators.

As we were discussing in practice, optimizing for recall is not always the best solution in terms of choosing the candidate generators and how much to extract from them - for example, you can learn to predict, whether the later-stage rankers will "like" the item from this candidate generator. The like can be defined as expected score, position in final ranking or binary marker that item is in top-N of all the candidates.

But we want to use it to estimate, how much items do we need to extract from CG - we can't just "rank" them using item-dependent features. Moreover, we only know user features, candidate generator information, context information (time, place, etc) - but it's actually enough, if you add one other feature: position of item in candidate list from CG. This way, before extracting any candidates, we can run the following procedure:
- Extract all the user and context features.
- Create a sample for each CG and fill it with CG features and user/context ones.
- Duplicate them and fill in the "position in cg" feature with all the possible options.
- Score and rerank.

Then we have a ranked list, where each sample contains a CG, from which we should extract another candidate. We can simply cutoff top-N of this list to get the number of candidates from each CG that we need to build the best N candidate list.

It's better to use a listwise ranking loss for this task, because candidates depend on each other inside each request - more on this will be on week04, so you can postpone the bonus task, but any additional knowledge isn't required to solve it.

In fact, this naive way will be pretty slow, but you can optimize it by not scoring a hundred candidates for each CG, but using [k-way merge](https://en.wikipedia.org/wiki/K-way_merge_algorithm) by iteratively scoring next candidate from CG, from which you pulled the last, or doing it in batches of ten for maximum speed, since extracting one candidate is not really standard operation. This is correct, if we assume that score is monotously decreasing, when position in candidate generator is increasing. Our main goal for this bonus will be to implement the model and see if the assumption is correct!

For more details (or a clearer explanation) you can read the description of the algorithm from one of it's authors [here](https://t.me/WazowskiRecommends/59) and [here](https://t.me/WazowskiRecommends/61).

### What do you have to do
For training mixigen, we need a ranker model - the code to compute the features and train a really simple catboost model. We also need a training dataset - logged candidate lists for several CGs with item and user features.

1. Train the ranker (and optionally add more features)
2. Train the models and create CGs
3. For each request, which has at least one positive item, in mixigen training subset call each CG and log top-100 candidates
4. For each logged candidate, join item and user features and score them using ranker model
5. Define the target for mixigen - whether the item was in top-N for the request.
6. Train the MixiGen model

### Solution

In [ ]:
# !cp -r recsys_course/week03/homework/train_cbm_ranker.py train_cbm_ranker

In [ ]:
import os
from catboost import CatBoostRanker, Pool

# user_features and item_features return dataframes with conversion and cart update rates by user and item
from train_cbm_ranker import train_cbm_ranker, user_features, item_features

In [ ]:
# we'll use a smaller subset of data, you can reduce it even further to fit in your RAM
data, _ = train_test_split(
    pl.read_parquet(f"{DATA_DIR}/train.parquet").sort("timestamp"),
    test_size=0.2,
    shuffle=False
)

train_models_data, train_ranker_data = train_test_split(
    data.sort("timestamp"), test_size=0.5, shuffle=False
)

# train and save the ranker model
if not os.path.exists("ranker.cbm"):
    ranker = train_cbm_ranker(train_models_data, train_ranker_data)
    ranker.save_model("ranker.cbm")
else:
    ranker = CatBoostRanker()
    ranker.load_model("ranker.cbm")

In [ ]:
# convert data to format for CG model training
mlm_format_data = (
    data
    .filter(pl.col("action_type") == "AT_CartUpdate")
    .select(
        pl.col("user_id"),
        pl.col("product_id").alias("item_id"),
        pl.col("request_id"),
        pl.lit(1).alias("rating"),
        pl.col("timestamp"),
    )
    .sort("timestamp")
)

train_models, _ = train_test_split(mlm_format_data, test_size=0.5, shuffle=False)

# we want to find the requests, where user added at least one item to cart
requests_for_mixigen_training = (
    train_ranker_data
    .filter(pl.col("action_type") == "AT_CartUpdate")
    .select("request_id", "user_id")
    .unique()
    .filter(
        pl.col("request_id").is_not_null()
        & pl.col("user_id").is_not_null()
    )
)

Train a set of 3-4 CG models (you can add your own, like random sampling, to experiment) and for each user in `user_for_mixigen_training` call each CG, log it's candidate list. Each item in list should contain `request_id`, `user_id`, `product_id`, `position_in_cg` and `source_cg`.

In [ ]:
models = {
    "ALS": ALS(),
    # TODO: add your CG models here

}

cgs = {
    # TODO: fill with ANN/KNN/etc CG instances
}

for model in models:
    model.fit(train_models)

    # TODO: train model and build CG from it


mixigen_train = []

for model in models:
    for user_id, request_id in requests_for_mixigen_training.iter_rows():

        # TODO: your code here

        # model.extract_candidates(user)
        pass

mixigen_train = pl.DataFrame(mixigen_train)

In [ ]:
assert "position_in_cg" in mixigen_train.columns
assert "source_cg" in mixigen_train.columns
assert "request_id" in mixigen_train.columns
assert "user_id" in mixigen_train.columns
assert "product_id" in mixigen_train.columns

Then, for each sample you should join the user and item features, computed on `train_models` subset of data and score them using you ranker model. After that you can order them within each request by score and set 1 as target, if item is in top-N, and 0 otherwise.

In [ ]:
N = 30

mixigen_targets = []

# TODO: your code here

mixigen_targets = pl.DataFrame(mixigen_targets)

assert "target" in mixigen_targets.columns
assert "request_id" in mixigen_targets.columns
assert "product_id" in mixigen_targets.columns

mixigen_train = (
    mixigen_train
    .join(mixigen_targets, on=("request_id", "product_id"), how="left")
    .sort("request_id")
)

# TODO: add cg or user features you want to use

The dataset we created is actually huge - we do not need that much data for our tasks, our features are way too dumb for that much data. Let's undersample by requests.

In [ ]:
sampled_requests = mixigen_train["request_id"].unique().sample(fraction=0.5)

sampled_mixigen_train = (
    mixigen_train
    .filter(pl.col("request_id").is_in(sampled_requests))
)

Now we can actually build the dataset for training and fit the model! We'll leave samples with up to N + 10 position in source CG, although they obviously won't be in top-N of ranking :)

In [ ]:
x_train, x_test = train_test_split(sampled_mixigen_train, test_size=0.2, shuffle=True)
x_train = x_train.sort("request_id")
x_test = x_test.sort("request_id")


train_pool = Pool(

    # TODO: fill numerical features, cat features, target and feature names

    # group is mandatory for ranking losses
    group_id=x_train["request_id"].cast(str).to_numpy(),
)
test_pool = Pool(

    # TODO: fill numerical features, cat features, target and feature names

    # group is mandatory for ranking losses
    group_id=x_test["request_id"].cast(str).to_numpy(),
)

model = CatBoostRanker(eval_metric="NDCG", iterations=100)
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)

We've trained the model! Now let's see if our assumption about the position in CG was correct - the score should decrease with bigger position.

The reference picture looks something like that:

<img src="mixigen.png" alt="drawing" width="600"/>

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plotting_data = x_test.with_columns(
    mixigen_score=model.predict(test_pool)
).select("cg", "rank", "mixigen_score")

plt.figure(figsize=(10, 6))
sns.lineplot(plotting_data.limit(10000).to_pandas(), x="rank", y="mixigen_score", hue="cg")
plt.xlabel("Position in the candidate generator")
plt.ylabel("Blending score")
plt.legend(title="Model")
plt.show()

In order to implement a fancy class with k-way merging of the candidate generators, we actually have to rewrite the candgens to support iteratively returning results instead of making huge queries, and current implementations won't be efficient with this - that's why we are doing only the model part.